In [111]:
from sklearn.datasets import load_wine
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (load_wine())

x = pd.DataFrame(datasets["data"])
y = pd.DataFrame(datasets["target"])

In [112]:
print(x.shape,y.shape) # (178, 13) (178, 1)
print(y[:5])

# y의 클래스 값 분류 + 불균형확인
print(np.unique(y)) # [0 1 2]
print(np.unique(y,return_counts=True)) # [0 1 2]

y = to_categorical(y)

(178, 13) (178, 1)
   0
0  0
1  0
2  0
3  0
4  0
[0 1 2]
(array([0, 1, 2]), array([59, 71, 48], dtype=int64))


In [113]:
# print(x.info())
# print(x.describe())
# 데이터 분리
x_train, x_test, y_train,y_test = train_test_split(x,y,
                                                   train_size=0.8,
                                                   shuffle = True,
                                                #  stratify 는 데이터 불균형을 해결해줌
                                                   stratify=y
                                                #    random_state=21
                                                   )
# print(x.columns)
# y 의 첫번째 칼럼을 원핫 인코딩 변환
# y_train[0] = tf.one_hot(y_train[0],3)
# y_test[0] = tf.one_hot(y_test[0],3)
print(y_train[:5])
print("shape : ",y_train.shape)

[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
shape :  (142, 3)


In [114]:
model = Sequential([
    Dense(50,activation="relu",input_shape=(13,)),
    Dense(40,activation="relu") ,
    Dense(40,activation="relu") ,
    Dense(30,activation="relu") ,
    Dense(30,activation="relu") ,
    Dense(20,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(3,activation="softmax")
])

In [115]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
min_delta=0.01, patience=5, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=100, batch_size=32, 
                 validation_split=0.2,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


Epoch 1/100
4/4 [==============================] - 0s 48ms/step - loss: 9.6447 - accuracy: 0.2389 - val_loss: 6.5938 - val_accuracy: 0.3103
Epoch 2/100
4/4 [==============================] - 0s 11ms/step - loss: 5.1753 - accuracy: 0.3363 - val_loss: 3.3800 - val_accuracy: 0.3103
Epoch 3/100
4/4 [==============================] - 0s 11ms/step - loss: 2.0674 - accuracy: 0.3717 - val_loss: 1.4900 - val_accuracy: 0.4138
Epoch 4/100
4/4 [==============================] - 0s 11ms/step - loss: 1.6170 - accuracy: 0.3805 - val_loss: 2.1585 - val_accuracy: 0.2759
Epoch 5/100
4/4 [==============================] - 0s 11ms/step - loss: 1.4071 - accuracy: 0.5310 - val_loss: 2.1210 - val_accuracy: 0.3103
Epoch 6/100
4/4 [==============================] - 0s 10ms/step - loss: 1.2986 - accuracy: 0.3717 - val_loss: 0.9481 - val_accuracy: 0.4483
Epoch 7/100
4/4 [==============================] - 0s 10ms/step - loss: 1.2251 - accuracy: 0.2832 - val_loss: 1.0465 - val_accuracy: 0.5172
Epoch 8/100
4/4 [===

In [116]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


2/2 [==============================] - 0s 4ms/step - loss: 0.6196 - accuracy: 0.7222
loss: 0.6195555329322815 
acc : 0.7222222089767456


In [117]:
from sklearn.metrics import accuracy_score
y_predict = model.predict(x_test)

# predict 의 값은 총 3개가 나오는데 softmax 특성상 sum 의 값은 1이 됨
print(y_predict[:5])

# argmax 는 x 를 최대값으로 만들어주는 입력을 구하는함수
y_predict = np.argmax(y_predict,axis=1)
# y_predict = to_categorical(y_predict,3)
print(y_predict[:5])
# y_test 를 원핫 인코딩 해제 해야함
y_test = np.argmax(y_test,axis=1)
print(y_test[:5])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print(acc)

[[3.7233719e-01 3.8855308e-01 2.3910968e-01]
 [1.0860028e-01 5.0814509e-01 3.8325465e-01]
 [9.9711156e-01 2.0479271e-03 8.4052258e-04]
 [6.7412198e-02 5.5603230e-01 3.7655556e-01]
 [1.2014381e-01 4.9699479e-01 3.8286141e-01]]
[1 1 0 1 1]
[1 1 0 1 1]
0.7222222222222222
